In [1]:
!pip install gensim

In [40]:
import pandas as pd
from datetime import datetime
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!wget -c http://nlp.stanford.edu/data/glove.6B.zip

--2020-08-04 12:56:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-04 12:56:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-04 12:56:28--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [4]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [5]:
url_train = 'https://raw.githubusercontent.com/fsicardir/datos-tp2/master/dataset/train.csv?token=AFVAIUW66UE3NA5X2SYXNPC7GHGJY'
url_test = 'https://raw.githubusercontent.com/fsicardir/datos-tp2/master/dataset/test.csv?token=AFVAIUUSBVEOOMDIFV4GU6C7GHGNK'

read_train = pd.read_csv(url_train)
read_test = pd.read_csv(url_test)

In [6]:
df_train = read_train[['id', 'text', 'target']]
df_test = read_test[['id', 'text']]

In [ ]:
# Limpiamos los datos de la forma usual
# Quitamos las urls
df_train['text'] = df_train['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)
df_test['text'] = df_test['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)

# Quitamos user mentions, signos de puntuación, hashtags y stopwords.
def clean_text(text):
    words = text.lower().split(' ')
    text = ' '.join([word for word in words if not word.startswith('@') and word not in stopwords.words('english')])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df_train['text'] = df_train['text'].apply(clean_text)
df_test['text'] = df_test['text'].apply(clean_text)


In [8]:
train_tweets = df_train['text'].tolist()
train_target = df_train['target']
test_tweets = df_test['text'].tolist()
len(train_tweets), len(train_target), len(test_tweets)

(7613, 7613, 3263)

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tweets)
vocabulary_size = len(tokenizer.word_index) + 1

In [11]:
sequences_train = tokenizer.texts_to_sequences(train_tweets)
len(sequences_train)

7613

In [12]:
# Sabemos que el tweet más largo en palabras tiene 25 palabras.
# Vamos a agregar el padding "al final"
max_padding = 25
padded_vecs_train = pad_sequences(sequences_train, maxlen=max_padding, padding='post')

In [13]:
# Cargamos los embeddings de GloVe.
glove_dict = {}
with open('glove.6B.100d.txt') as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], dtype='float32')
      glove_dict[word] = vector

In [14]:
# Ahora vamos a crear una matriz que tendrá los embeddings de GloVe
# correspondientes a cada palabra de nuestro vocabulario.
# Esto se lo pasaremos como pesos a la capa de Embedding del modelo a entrenar.
embedding_dim = 100
embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_dict.get(word, None)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [15]:
# Vamos a dividir el set de entrenamiento para obtener un set de validación
# y así comenzar a entrenar nuestro modelo.
X_train, X_test, y_train, y_test = train_test_split(padded_vecs_train, train_target, test_size=0.2, random_state=17)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6090, 25), (6090,), (1523, 25), (1523,))

In [111]:
# Ahora creemos nuestro modelo
model = Sequential()
emb_layer = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_padding, trainable=False)

model.add(emb_layer)
model.add(Conv1D(128, 5, activation='relu', padding='same'))
model.add(Conv1D(128, 5, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 25, 100)           1540300   
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 25, 128)           64128     
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 25, 128)           82048     
_________________________________________________________________
global_max_pooling1d_19 (Glo (None, 128)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_33 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)               

In [112]:
epochs = 10
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
191/191 [==============================] - 4s 21ms/step - loss: 0.4912 - accuracy: 0.7749 - val_loss: 0.4618 - val_accuracy: 0.7919
Epoch 2/10
191/191 [==============================] - 4s 20ms/step - loss: 0.4032 - accuracy: 0.8281 - val_loss: 0.4426 - val_accuracy: 0.8030
Epoch 3/10
191/191 [==============================] - 4s 20ms/step - loss: 0.3494 - accuracy: 0.8501 - val_loss: 0.4593 - val_accuracy: 0.8024
Epoch 4/10
191/191 [==============================] - 4s 20ms/step - loss: 0.2589 - accuracy: 0.8974 - val_loss: 0.4884 - val_accuracy: 0.7932
Epoch 5/10
191/191 [==============================] - 4s 20ms/step - loss: 0.1898 - accuracy: 0.9322 - val_loss: 0.5956 - val_accuracy: 0.7768
Epoch 6/10
191/191 [==============================] - 4s 20ms/step - loss: 0.1353 - accuracy: 0.9555 - val_loss: 0.7104 - val_accuracy: 0.7800
Epoch 7/10
191/191 [==============================] - 4s 20ms/step - loss: 0.1060 - accuracy: 0.9631 - val_loss: 0.6826 - val_accuracy: 0.7886

In [113]:
def metrics(predictions, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(f'Verdaderos Negativos: {tn}')
    print(f'Falsos Negativos: {fn}')
    print(f'Verdaderos Positivos: {tp}')
    print(f'Falsos Positivos: {fp}')
    print()
    print(f'precision score: {precision_score(y_test, predictions)}')
    print(f'recall score: {recall_score(y_test, predictions)}')
    print(f'f1 score: {f1_score(y_test,  predictions)}')

In [114]:
preds = model.predict_classes(X_test)

metrics(preds, y_test)

Verdaderos Negativos: 727
Falsos Negativos: 181
Verdaderos Positivos: 486
Falsos Positivos: 129

precision score: 0.7902439024390244
recall score: 0.7286356821589205
f1 score: 0.7581903276131046


In [115]:
# Entrenamos la red con el total del set de datos y luego hacemos una predicción para la competencia.
sequences_test = tokenizer.texts_to_sequences(test_tweets)
padded_vecs_test = pad_sequences(sequences_test, maxlen=max_padding, padding='post')
model.fit(padded_vecs_train, train_target, epochs=epochs)

Epoch 1/10
238/238 [==============================] - 4s 19ms/step - loss: 0.2297 - accuracy: 0.9234
Epoch 2/10
238/238 [==============================] - 4s 19ms/step - loss: 0.1228 - accuracy: 0.9519
Epoch 3/10
238/238 [==============================] - 4s 19ms/step - loss: 0.0886 - accuracy: 0.9637
Epoch 4/10
238/238 [==============================] - 4s 19ms/step - loss: 0.0742 - accuracy: 0.9679
Epoch 5/10
238/238 [==============================] - 5s 19ms/step - loss: 0.0633 - accuracy: 0.9711
Epoch 6/10
238/238 [==============================] - 4s 19ms/step - loss: 0.0579 - accuracy: 0.9735
Epoch 7/10
238/238 [==============================] - 5s 20ms/step - loss: 0.0537 - accuracy: 0.9744
Epoch 8/10
238/238 [==============================] - 5s 20ms/step - loss: 0.0549 - accuracy: 0.9744
Epoch 9/10
238/238 [==============================] - 5s 20ms/step - loss: 0.0860 - accuracy: 0.9610
Epoch 10/10
238/238 [==============================] - 4s 19ms/step - loss: 0.0988 - accura

In [116]:
kaggle_preds = model.predict_classes(padded_vecs_test)

result = df_test[['id']]
result['target'] = kaggle_preds
result.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [117]:
now = datetime.now(tz=None).strftime('%Y-%m-%dT%H:%M:%S')
result.to_csv(f'prediction_{now}.csv', encoding='utf-8', index=False)